# Introduction

In this notebook I will create a chatbot to help resolve issues with a fictional software service using the ChatGPT API. The gpt-4o model will be given contextual data about the specific features and possible errors of this software. This will help it answer difficult questions from confused customers similarly to how a live person would.

In [2]:
from openai import OpenAI

client = OpenAI(api_key = 'YOUR KEY HERE')

You will need an API key and the OpenAI library

# Contextual Data

In [3]:
context_data = {
    "services": {
        "Service A": "Provides advanced **data analysis tools** that help businesses gain insights from their data. Includes visualization, predictive analytics, and integration with external APIs.",
        "Service B": "Offers **cloud storage and file syncing** across all your devices. With **automatic backup** and **real-time syncing**, your data is always up-to-date wherever you go.",
        "Service C": "Includes **collaborative document editing**, allowing multiple users to edit, comment, and track changes in documents in real-time. Perfect for team collaboration and project management.",
        "Service D": "Features **automated task scheduling and reminders** to help you stay organized. With support for custom workflows and integration with third-party apps, it ensures nothing gets missed.",
        "Service E": "Provides **API integrations** for connecting your software stack to third-party tools. Enables seamless data exchange and automation across platforms for maximum efficiency.",
        "Service F": "Offers **customizable dashboards** that allow you to visualize your business performance in real-time, tailored to your specific KPIs and metrics.",
    },
    
    "error_codes": {
        "E001": "File not found. Please check the **file path** and ensure the file exists.",
        "E002": "Sync failed. Ensure you're connected to the **internet** and that your connection is stable.",
        "E003": "Permission denied. Verify **user roles** and permissions to access this resource.",
        "E004": "**Timeout** error. The operation took too long to complete. Try again later or check the server status.",
        "E005": "Data format error. The file format is not compatible. Please upload a supported file type.",
        "E006": "Authentication failed. Please check your **login credentials** and try again.",
        "E007": "Connection error. Unable to reach the server. Please check your **network connection**.",
    },

    "plans": {
        "Basic": "Provides access to core features such as file syncing, 5 GB of storage, and basic document collaboration. Ideal for personal use or small teams.",
        "Pro": "Unlocks advanced features such as unlimited storage, API integrations, and premium customer support. Perfect for growing teams and businesses.",
        "Enterprise": "Offers customizable solutions for large organizations, including enhanced security, dedicated support, and tailored workflows. Includes everything in the Pro plan, plus more.",
    }
}

This dictionary with services, error codes and plans was generated by ChatGPT for the purpose of this exercise. This data format is perfect for this purpose as it enables gives the model the context for potential questions. Notice how every category is interrelated. For example only some plans give access to certain services and some errors only arise from the usage of certain services.

# Chat Function

In [4]:
def get_gpt4_response(user_input, conversation_history, context_data):
    context = (
        "You are a helpful customer support assistant. Use the following information to answer queries:\n"
        "Services:\n" +
        "\n".join([f"{key}: {value}" for key, value in context_data["services"].items()]) +
        "\n\nError Codes:\n" +
        "\n".join([f"{key}: {value}" for key, value in context_data["error_codes"].items()]) +
        "\n\nplans:\n" +
        "\n".join([f"{key}: {value}" for key, value in context_data["plans"].items()]) +
        "\n\n"
    )

    messages = [{"role": "system", "content": context}]
    messages += [{"role": "user", "content": msg} for msg in conversation_history]
    messages.append({"role": "user", "content": user_input})
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )
    
    assistant_response = response.choices[0].message.content
    conversation_history.append(user_input)
    conversation_history.append(assistant_response)
    
    return assistant_response

This function will tell the chatbot to act as a customer support assistant and also to keep in mind the contextual data needed to solve these issues. Additional messages will be added to the context to keep track of the conversation until erased.

# Sample Questions

lets test this chatbot by giving it some questions and seeing how it responds.

In [4]:
conversation_history = []

user_input = "Tell me about your Enterprise plan"
response = get_gpt4_response(user_input, conversation_history, context_data)
print(response)

Our Enterprise plan offers customizable solutions designed specifically for large organizations. It includes everything in the Pro plan, plus additional features such as enhanced security, dedicated support, and tailored workflows. This plan is ideal for businesses that require a high level of customization and support to meet their specific needs. If you're looking for a comprehensive plan that offers unlimited storage, API integrations, premium customer support, and more, the Enterprise plan could be the right choice for your organization.


The first question is pretty straightforward. The bot is asked about a specific plan and it gives an answer. Even expanding a little bit on what was given to it in the contextual data.

In [5]:
conversation_history = []

user_input = "E006"
response = get_gpt4_response(user_input, conversation_history, context_data)
print(response)

The error code E006 indicates that authentication failed. Please check your login credentials and try again. Make sure that you are entering the correct username and password. If you continue to experience issues, you may want to reset your password or contact customer support for further assistance.


Another simple example. An error code is given and the bot gives a summary of the error as well as how it could be resolved.

In [6]:
conversation_history = []

user_input = "I am having issues with the cloud storage"
response = get_gpt4_response(user_input, conversation_history, context_data)
print(response)

I'm sorry to hear you're having trouble with the cloud storage. Could you provide more details about the issue you're experiencing? Are you encountering any error codes or receiving specific error messages? Understanding more about the problem can help in providing you with the best solution.


In [7]:
user_input = "Error two"
response = get_gpt4_response(user_input, conversation_history, context_data)
print(response)

Error Code E002 indicates a "Sync failed" issue. Here are a few steps you can take to resolve this problem:

1. **Check Your Internet Connection**: Ensure you have a stable internet connection. You might want to try connecting to a different network to see if the issue persists.

2. **Restart Syncing Services**: Sometimes, simply restarting your device or the application can help reset the connection and fix the syncing issue.

3. **Check for Updates**: Make sure that your application and device are running the latest software updates, as updates often include fixes for known issues.

4. **Check Storage Capacity**: Ensure you have not exceeded your storage limit, especially if you are on the Basic plan with 5 GB of storage.

5. **Try Again Later**: Occasionally, the issue could be due to temporary server problems on our end. If possible, wait a while and try syncing again.

If the problem continues, please feel free to reach out with additional details or contact customer support for f

This example is a multi-part question. Some users might begin with asking for help first before saying what happened. Notice that here the user asks about 'error two' rather than giving the error code.

In [8]:
conversation_history = []

user_input = "what error did I just say I had?"
response = get_gpt4_response(user_input, conversation_history, context_data)
print(response)

I'm sorry, but I don't have access to previous interactions or the ability to recall past conversations. Could you please let me know the error you're encountering, and I'll do my best to assist you?


When talking to a new user, the bot won't be confused by the previous query.

In [9]:
conversation_history = []

user_input = "I am having issues with service D"
response = get_gpt4_response(user_input, conversation_history, context_data)
print(response)

I'm sorry to hear you're having issues with Service D, which features automated task scheduling and reminders. Could you provide more details about the specific problem you're experiencing? This will help me better understand the issue and provide you with the appropriate assistance.


In [10]:
user_input = "error 1"
response = get_gpt4_response(user_input, conversation_history, context_data)
print(response)

Error Code E001 indicates that a "File not found" issue has occurred. For Service D, which involves task scheduling and reminders, this error might be unusual, as it is primarily related to file operations. I recommend checking the following:

1. **Verify File Paths**: Ensure that any files associated with your tasks or workflows are correctly referenced and exist in the specified location.

2. **Check Integration Settings**: If your tasks are linked to external files or apps, make sure those integrations are properly set and the files are accessible.

If the issue persists or if you need further assistance, feel free to provide more details so I can help you troubleshoot further!


Here the bot shows that it understands how the errors and services are interconnected by saying that this error doesn't usually occur for this service. Nuances like this are crucial when solving tech issues.

In [11]:
conversation_history = []

user_input = "Why can't I access API integration?"
response = get_gpt4_response(user_input, conversation_history, context_data)
print(response)

To access API integration, you need to be subscribed to the Pro plan or higher. The Basic plan does not include API integrations. If you are on the Pro or Enterprise plan and still cannot access the API integration, you might want to check if you have the necessary permissions assigned to your user role or if there are any specific error messages indicating why access might be restricted (such as E006: Authentication failed).


Another example where the bot is aware that some services are not present for certain plans and even references a potential error.

In [5]:
conversation_history = []

user_input = "Its not working!"
response = get_gpt4_response(user_input, conversation_history, context_data)
print(response)

I'm sorry to hear that you're experiencing issues. Could you please provide more details about the problem you're facing? This will help me assist you more effectively. For example, are you seeing any error messages, and if so, what do they say? Which service are you having trouble with?


In [6]:
user_input = "the callendr"
response = get_gpt4_response(user_input, conversation_history, context_data)
print(response)

It looks like you might be referring to an issue with the calendar feature, which could be related to our Service D: automated task scheduling and reminders. Can you provide more details about the problem you're experiencing? Are you encountering any specific error messages or issues with scheduling tasks and reminders? Any additional information will help me assist you better.


In [7]:
user_input = "says i dont have permission??"
response = get_gpt4_response(user_input, conversation_history, context_data)
print(response)

If you’re receiving a message about permissions, it might be related to error code E003: Permission denied. Here are a few steps you can take to resolve this issue:

1. **Check User Roles and Permissions**: Ensure that your user account has the necessary permissions to access and modify the calendar or scheduling feature. You may need to contact your account administrator if permissions were recently changed.

2. **Review Access Controls**: If your organization uses role-based access controls, verify that your user role is assigned the correct permissions for the operations you need to perform.

3. **Update Settings**: If you have access to settings or management features, ensure your permissions are correctly configured for task scheduling and reminders.

If you’re unsure how to verify your permissions or if the issue persists, please provide more details about your account setup or contact support for further assistance.


In [8]:
user_input = "okay just updated its good now thank you"
response = get_gpt4_response(user_input, conversation_history, context_data)
print(response)

Great to hear that the issue has been resolved after updating! If you have any more questions or need further assistance, feel free to reach out. Have a wonderful day!


Here despite the the user's confusion and spelling errors the bot still understands and is able to guide them to resolving the issue.

# Chatbot Widget

Here is an example of a more user friendly way to interact with the chatbot using a widget.

In [11]:
import ipywidgets as widgets
from IPython.display import display, clear_output

conversation_history = []

def handle_user_input(change):
    user_input = input_box.value.strip()
    if not user_input:
        return
    if user_input.lower() == "exit":
        with output:
            print("Chatbot: Goodbye!")
        input_box.disabled = True
        return
    with output:
        print(f"You: {user_input}")
    chatbot_response = get_gpt4_response(user_input, conversation_history, context_data)
    with output:
        print(f"Chatbot: {chatbot_response}")
    conversation_history.append(user_input)
    conversation_history.append(chatbot_response)
    input_box.value = ""

input_box = widgets.Text(
    placeholder="Type your question here...",
    description="You:",
    layout=widgets.Layout(width="100%")
)

output = widgets.Output()
input_box.continuous_update = False
input_box.observe(handle_user_input, names="value")

print('Hello!\nPlease tell me about the issue you are having.\nTry to include the error code if present.\nType "exit" when you are done.')
display(widgets.VBox([output, input_box]))

Hello!
Please tell me about the issue you are having.
Try to include the error code if present.
Type "exit" when you are done.


In [13]:
output

Output(outputs=({'output_type': 'stream', 'text': 'You: How do I add a comment to my manager\'s edit?\nChatbot…

In this example the widget was used to ask a series of questions. The user started by asking about how to perform an action then the bot was able to reference the name of the service, the error code and how to finally solve the problem.

# Conclusion

With just a little bit of contextual information, the chatbot was able to answer questions of various complexity. Some were handled surprisingly well knowing ChatGPT's issues with multi-step questions and word problems. This can be adapted to different services and products as long as enough context is given. 

Thank you for reading!